# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [4]:
# set up environment
system_prompt = "You are a technical assistant. Your will receive some technical code snippits and explain them in detail."

In [5]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [9]:
# Get gpt-4o-mini to answer, with streaming
def get_answer_gpt():
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [10]:
get_answer_gpt()

The code snippet you provided is a Python expression that utilizes the `yield from` statement along with a set comprehension. Let's break it down step by step:

1. **`books`**: This implies that `books` is likely a list (or any iterable) of dictionaries, where each dictionary represents a book. Each book dictionary presumably contains various key-value pairs, one of which is `"author"`.

2. **Set Comprehension**: The expression `{book.get("author") for book in books if book.get("author")}` is a set comprehension. 

   - It iterates over each `book` in the `books` collection.
   - The `book.get("author")` method is called to retrieve the value associated with the `"author"` key for each `book`. Using `get()` is beneficial because it will return `None` instead of throwing an error if the `"author"` key doesn’t exist in a particular `book`.
   - The `if book.get("author")` conditional ensures that only books that have a valid (non-`None`) author name are included in the set. This means that if the author is not specified or is `None`, that book will be skipped.

3. **Result of the Set Comprehension**: The set comprehension will produce a set of unique author values found in the `books`. Since it’s a set, it automatically handles duplicates – if multiple books have the same author, that author will only appear once in the resulting set.

4. **`yield from` Statement**: The `yield from` statement is used within generator functions to yield all values from an iterable. It simplifies the process of yielding values from sub-generators.

   - In this context, the code is effectively using `yield from` to yield each unique author from the set generated in the previous step.

### Summary

In summary, the entire expression:
python
yield from {book.get("author") for book in books if book.get("author")}

does the following:
- It generates a set of unique author names from a list of books, avoiding any `None` values (i.e., books without an author).
- The authors are then yielded one by one, suggesting that this code is likely part of a generator function. This allows the caller to iterate over unique authors efficiently.

This design is useful in scenarios where you want to process or work with unique items derived from a larger collection while maintaining memory efficiency and cleaner code through the use of generators.

In [13]:
# Get Llama 3.2 to answer

# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL_LLAMA,
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
    stream=True
    )
    
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)
        
print(response.choices[0].message.content)

NameError: name 'stream' is not defined

In [14]:
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

def get_ans_llama():
    stream = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [15]:
get_ans_llama()

**Iterating over Nested Data Structures using `yield from`**

This line of code uses the `yield from` syntax to iterate over a nested data structure. Let's break it down:

*   `{ book.get("author") for book in [books] }`: This is a generator expression that iterates over each item in `books`, retrieves the value associated with the key `"author"` using the `get()` method, and yields those values.
*   `yield from { ... }`: The outer syntax is `yield from`, which is used to delegate to another iterable or generator.

**What does it do?**

This code iterates over the list of books (`books`), retrieving only the `"author"` values associated with each book. Here's a step-by-step explanation:

1.  The code iterates over each item in the `books` list, typically accessed through an object like a dictionary or a pandas DataFrame (in real Python applications).
    python
for book in books:


2.  For each book, it uses tuple unpacking (`book.get("author")`) to retrieve the value associated with the key `"author"`, if present.
    python
book.get("author")


   This generates an iterable containing all the authors (if any) from each book.

3.  Then, `yield from` delegates this generator to a larger context; it essentially merges their iterables into a single sequence. As such, its output is another iterator that yields values, one from each of those smaller generators.
    python
yield from { ... }


**When would you use this approach?**

This syntax is useful when you need to iterate over the results of multiple, independent iterators (such as database queries or file processes). You can apply it in many scenarios:

*   **Data processing:** When you have multiple data sources (like CSV files) and want to combine their contents.
*   **Database queries:** If a single query retrieves data from multiple tables and need to yield values from each table separately.

Here is a more complete Python code example which uses these concepts, demonstrating its applicability:

python
import pandas as pd

# Generating nested dataset by merging three separate CSV files into one, each having 'books' key.
frames = [pd.DataFrame(columns=["title", "author"]), 
          pd.DataFrame([{"book1": {"title": "Book 1 - Volume 1 of 2","author": "Author A1"},
                          "book1": {"title": "Book 1 - Volume 2 of 2", "author":"Author A2"},{"book2":{"title": "Volume B of 3 of 6", "author": "Aurora"}],
                        "books":{"volume_3_Authors":["Author B"}]}]),
          pd.DataFrame(columns=["Title", "Author"])]
dataframe = pd.concat(frames, ignore_index=True)
# Iterate and extract results in a generator form from dataframe or a list of objects stored in separate files
for title, book_author in [{book: "Book Name"}.get(key) for key, book in dataframe.iterrows()] : yield(book_author)

This code example shows how to process the data returned by an external query (a SQL database). It yields results one at a time as needed.